In [35]:
import glob
import pandas as pd
import numpy as np
import os
import cv2

Things you need to run the code:
1. All your cam1 .h5 files in one folder. This will be the videos folder under the main DLC Project folder.
2. Same as above for all your cam 2 .h5 files.

Basically have all cam1 and all cam 2 h5 files in two different folders. 

3. Create a new folder where you want all your dltdv8 format points to be saved.

To run the code:
1. Import all the libraries and run the DLCh5_DLTcsv definition.

2. set the following variables
'all_cam1_paths' is the folder where all your cam1 .h5 files are.
'all_cam2_paths' is the folder where all your cam2 .h5 files are.
'output_path' is the folder where all your dlt coordinates will be dumped.

3. RUNNN

IMPORTANT: 
In creating the directory and saving the files part, you need to mention, which characters to pick from the cam1 path which will be different for every project. Change it accordingly. Use some online tool. try this https://www.lettercount.com/


In [36]:
def DLCh5_DLTcsv(Cam1_h5_path, Cam2_h5_path, output_path):
    """
    Function to extract the labelled points into a csv file for DLTdv8
    and to generate dummy offset, xyzpts and xyzres.csv files 

    Cam1_h5_path: A string of path to the h5 file of camera 1

    Cam2_h5_path: A string of path to the h5 file of camera 2

    """
    Cam1_h5 = pd.read_hdf(Cam1_h5_path) #Reading the h5 file from camera 1
    Cam2_h5 = pd.read_hdf(Cam2_h5_path) #Reading the h5 file from camera 2
    

    #Making a dummy offsets file
    offsets = pd.DataFrame(
        columns=['cam1_offset','cam2_offset'],
        index = np.array(Cam1_h5.index)
        )
    offsets['cam1_offset'] = np.zeros(shape = len(Cam1_h5))
    offsets['cam2_offset'] = np.zeros(shape = len(Cam2_h5))
    offsets = offsets.round(6) #Do I need this?

    #Making a new Dataframe to save it as csv
    xypts = pd.DataFrame(
        columns=['pt1_cam1_X', 'pt1_cam1_Y', 'pt1_cam2_X', 'pt1_cam2_Y']
        )
    xypts[xypts.columns[0]] = np.array(Cam1_h5[Cam1_h5.columns[0]])
    xypts[xypts.columns[1]] = np.array(Cam1_h5[Cam1_h5.columns[1]])
    xypts[xypts.columns[2]] = np.array(Cam2_h5[Cam2_h5.columns[0]])
    xypts[xypts.columns[3]] = np.array(Cam2_h5[Cam2_h5.columns[1]])
    xypts = xypts.round(6)

    #Making a new Dataframe with likelihoods of points
    #Replicating likelihood columns for X and Y of each point
    likelihood = pd.DataFrame(
        columns = ['pt1_cam1_X', 'pt1_cam1_Y', 'pt1_cam2_X', 'pt1_cam2_Y']
    )
    likelihood[likelihood.columns[0]] = np.array(Cam1_h5[Cam1_h5.columns[2]])
    likelihood[likelihood.columns[1]] = np.array(Cam1_h5[Cam1_h5.columns[2]])
    likelihood[likelihood.columns[2]] = np.array(Cam2_h5[Cam2_h5.columns[2]])
    likelihood[likelihood.columns[3]] = np.array(Cam2_h5[Cam2_h5.columns[2]])


    #Filtering based on likelihood from DLC
    pvalue = 0.95
    cutoff = likelihood[likelihood.columns[:]] > pvalue
    likelihood_filtered_xypts = xypts.mul(cutoff)
    likelihood_filtered_xypts.replace(0, np.nan, inplace=True)

    #Making a dummy xyzpts file
    xyzpts = pd.DataFrame(
        np.nan,
        columns=['pt1_X', 'pt1_Y', 'pt1_Z',],
        index = np.array(Cam1_h5.index)
        )
    
    #Making a dummy xyzres file
    xyzres = pd.DataFrame(
        np.nan,
        columns=['pt1_dltres'],
        index = np.array(Cam1_h5.index)
        )
    

    #Creating the directory and saving the csv files in the dir
    cwd = os.getcwd()
    file = Cam1_h5_path
    file_dir = output_path
    offsets.to_csv(file_dir+f'\{file[77:86]}_offsets.csv',index = False)
    xypts.to_csv(file_dir+f'\{file[77:86]}_xypts.csv',index = False, na_rep='NaN')
    likelihood.to_csv(file_dir+f'\{file[77:86]}_likelihood.csv',index = False)
    xyzpts.to_csv(file_dir+f'\{file[77:86]}_xyzpts.csv',index = False, na_rep='NaN')
    xyzres.to_csv(file_dir+f'\{file[77:86]}_xyzres.csv',index = False, na_rep='NaN')

    #Saving likelihood xypts and the supporting files
    offsets.to_csv(file_dir+f'\{file[77:86]}_likelihood_filtered_offsets.csv',index = False)
    likelihood_filtered_xypts.to_csv(file_dir+f'\{file[77:86]}_likelihood_filtered_xypts.csv',index = False, na_rep='NaN')
    xyzpts.to_csv(file_dir+f'\{file[77:86]}_likelihood_filtered_xyzpts.csv',index = False, na_rep='NaN')
    xyzres.to_csv(file_dir+f'\{file[77:86]}_likelihood_filtered_xyzres.csv',index = False, na_rep='NaN')

In [37]:
all_cam1_paths=glob.glob(r"E:\transparent chamber 3\transapent chamber 3 20654-Mitali-2022-12-16\videos\*.h5")
all_cam2_paths=glob.glob(r"E:\transparent chamber 3\transparent chamber 3 16024-Mitali-2022-12-16\videos\*.h5")
output_path = (r"E:\transparent chamber 3\dlc2dlt")

allfiles = pd.DataFrame(list(zip(all_cam1_paths, all_cam2_paths)), columns = ["cam1","cam2"])

for vid_no in allfiles.index:
    DLCh5_DLTcsv(allfiles['cam1'][vid_no], allfiles['cam2'][vid_no],output_path)
    